#Download libraries

In [1]:
!apt install python3-libtorrent
!python -m pip install lbry-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libtorrent-rasterbar2.0
The following NEW packages will be installed:
  libtorrent-rasterbar2.0 python3-libtorrent
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 2,098 kB of archives.
After this operation, 7,907 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libtorrent-rasterbar2.0 amd64 2.0.5-5 [1,501 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python3-libtorrent amd64 2.0.5-5 [597 kB]
Fetched 2,098 kB in 0s (8,142 kB/s)
Selecting previously unselected package libtorrent-rasterbar2.0:amd64.
(Reading database ... 121753 files and directories currently installed.)
Preparing to unpack .../libtorrent-rasterbar2.0_2.0.5-5_amd64.deb ...
Unpacking libtorrent-rasterbar2.0:amd64 (2.0.5-5) ...
Selecting previously unselected package python3-li

# Mount Google Drive

In [2]:

from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Method 1 : Download from torrent file
Note : Run Download block post uploading

In [ ]:
from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Torrent",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))


# Method 2 : Download from magnetic link

Note : Run Download block post uploading

In [3]:
params = {"save_path": "/content/drive/My Drive/Torrent"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )

Enter Magnet Link Or Type Exit: magnet:?xt=urn:btih:6184EBEF59790820D44C39811820D0CC45359184&dn=Ataraxia+-+Torments+of+Reality+%282024%29&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2Fopen.stealth.si%3A80%2Fannounce&tr=udp%3A%2F%2Ftracker.torrent.eu.org%3A451%2Fannounce&tr=udp%3A%2F%2Ftracker.cyberia.is%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.dler.org%3A6969%2Fannounce&tr=udp%3A%2F%2Fexodus.desync.com%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.tiny-vps.com%3A6969%2Fannounce&tr=udp%3A%2F%2Fbt1.archive.org%3A6969%2Fannounce&tr=udp%3A%2F%2Fexplodie.org%3A6969%2Fannounce&tr=udp%3A%2F%2Fopen.demonii.com%3A1337%2Fannounce&tr=udp%3A%2F%2Ftracker.coppersurfer.tk%3A6969%2Fannounce&tr=udp%3A%2F%2F9.rarbg.to%3A2710%2Fannounce&tr=udp%3A%2F%2Ftracker.leechers-paradise.org%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.open-internet.nl%3A6969%2Fannounce&tr=udp%3A%2F%2Fopen.demonii.si%3A1337%2Fannounce&tr=udp%3A%2F%2Ftracker.pirateparty.gr%3A6969%2Fannounce&tr=udp%3A%2F%2Fdenis.stal

# Download

In [4]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close()
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)


FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

Ataraxia - Torments of Reality (2024) complete
